# Cofactor swap analysis

In [1]:
# Packages
#from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield
#from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
#from cobra.io import read_sbml_model, write_sbml_model
#from cobra import Reaction, Metabolite

In [2]:
# Loading of the model
model_orig = read_sbml_model('Sco-GEM.xml')
model = model_orig.copy()

In [3]:
# Calculate max biomass and undecyl prodigiosin (RED) production
model.solver = 'glpk'
with model:
    base_max_growth = model.slim_optimize()
    print("Max biomass growth:", base_max_growth)
    model.objective = model.reactions.DM_RED_c
    base_max_production = model.slim_optimize()
    print("Max RED production:", base_max_production)

Max biomass growth: 0.07461812924636277
Max RED production: 0.13190476190476172


In [4]:
# Set biomass boundaries
biomass = model.reactions.BIOMASS_SCO_tRNA.lower_bound = 0.045
# Set model objective towards undecyl prodigiosin biosynthesis
demand = model.reactions.get_by_id("DM_RED_c")
model.objective = demand

In [5]:
# Max biomass and undecyl prodigiosin production after objective change
with model:
    base_max_growth = model.slim_optimize()
    print("Max biomass growth:", base_max_growth)
    model.objective = model.reactions.DM_RED_c
    base_max_production = model.slim_optimize()
    print("Max RED production:", base_max_production)

Max biomass growth: 0.05254099064353999
Max RED production: 0.052540990643539294


In [6]:
# Product yield calculation
py = product_yield(model.reactions.DM_RED_c, model.reactions.EX_glc__D_e)

In [7]:
optswap = CofactorSwapOptimization(model = model, objective_function = py)

In [8]:
# Co-factor swap targets
%time
result = optswap.run(max_size=10)
result

Wall time: 0 ns
Starting optimization at Fri, 25 Nov 2022 16:48:53


HBox()

Finished after 00:01:16


,index,targets,fitness
0,0,"(GCALDD,)",0.273651
1,1,"(LIPOS,)",0.273651
2,2,"(3OARai70,)",0.273651
3,3,"(DIOP5OR,)",0.273651
4,4,"(CPKS3,)",0.273651
5,5,"(GLUSx, GLUSy, EAR60y, EARi180x)",0.273651
6,6,"(2ABZMOy,)",0.273651
7,7,"(REDS7,)",0.273651


Now we can check the effect of the co-factor swap.

In [9]:
# Max biomass and undecyl prodigiosin production after objective change
with model:
    swap_max_growth = model.slim_optimize()
    print("Max biomass growth:", swap_max_growth)
    model.objective = model.reactions.DM_RED_c
    swap_max_production = model.slim_optimize()
    print("Max RED production:", swap_max_production)

Max biomass growth: 0.052541070493969486
Max RED production: 0.05254107049396962


In [10]:
# Calcualte the % change in max biomass growth and undecyl prodigiosin production
print("Change in max growth:", round((swap_max_growth-(base_max_growth))/(base_max_growth) * 100, 4), "%")
print("Change in max production:", round((swap_max_production-(base_max_production))/(base_max_production) * 100, 4), "%")

Change in max growth: 0.0002 %
Change in max production: 0.0002 %
